In [41]:
%run CFBmodel/import_modules.ipynb
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = 'MyjBpIrnMbCmsxXJD6zmiOCWqjB3p+7Icq5PRSrN82IXeOppkoEWybx1SmRryxT+'
configuration.api_key_prefix['Authorization']='Bearer'
api_instance = cfbd.CoachesApi(cfbd.ApiClient(configuration))
# game_id=game_id  season_type=season_type, team=team, home=home, away=away, conference=conference)

In [47]:
coaches24 = api_instance.get_coaches(year=2024)


In [48]:
coaches24[0].seasons

[{'games': 0,
  'losses': 0,
  'postseason_rank': None,
  'preseason_rank': None,
  'school': 'Ohio',
  'sp_defense': 28.1,
  'sp_offense': 17.5,
  'sp_overall': -10.8,
  'srs': -10.7,
  'ties': 0,
  'wins': 0,
  'year': '2024'}]

In [52]:
import pandas as pd

def get_coaches(start):
    combined_data = []  # List to collect game data

    for j in range(start, 2025):  # Loop through weeks
        coach_data = api_instance.get_coaches(year=j)

        for coach in coach_data:
            # Compile all data into a single dictionary for each line
            game_info = {
                    'first_name': coach.first_name,
                    'hire_date': coach.hire_date,
                    'season_type': coach.last_name,
                    'year': j,
                     'games': coach.seasons[0].games,
                   'losses': coach.seasons[0].losses,
                   'postseason_rank': coach.seasons[0].postseason_rank,
                   'preseason_rank': coach.seasons[0].preseason_rank,
                   'school': coach.seasons[0].school,
                   'sp_defense':coach.seasons[0].sp_defense,
                   'sp_offense': coach.seasons[0].sp_offense,
                   'sp_overall': coach.seasons[0].sp_overall
                }
            combined_data.append(game_info)
            i=0
            for season in coach.seasons:
                coach_info = {
                     'games{}'.format(i): coach.seasons[0].games,
                   'losses{}'.format(i): coach.seasons[0].losses,
                   'postseason_rank{}'.format(i): coach.seasons[0].postseason_rank,
                   'preseason_rank{}'.format(i): coach.seasons[0].preseason_rank,
                   'school{}'.format(i): coach.seasons[0].school,
                   'sp_defense{}'.format(i):coach.seasons[0].sp_defense,
                   'sp_offense{}'.format(i): coach.seasons[0].sp_offense,
                   'sp_overall{}'.format(i): coach.seasons[0].sp_overall
                }
                i=i+1
                combined_data.append(coach_info)  # Append the dictionary to the list

    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(combined_data)
    return df

In [55]:
coachHist=get_coaches(start=1980)

In [60]:
coachHist.sp_offense0.value_counts()

sp_offense0
32.0    57
34.2    52
27.8    52
24.5    48
29.9    47
        ..
43.7     1
46.4     1
49.6     1
13.9     1
12.4     1
Name: count, Length: 410, dtype: int64

In [61]:
con=sqlite3.connect('CFBModel/collegeFootball.db')
cursor_obj= con.cursor()
cursor_obj.execute("DROP TABLE IF EXISTS coachHist")
coachHist.to_sql(name='coachHist',con=con)

10796

In [62]:
con.commit()
con.close()